**Item 46: Define class with get and set method which is resuable.**
- we use example of Grade class with get and set method combine with Example class with use multiple Grad class as it attribute
- we also discuss about the error of exam instance where two instance but attribute reference to the same grade instance
- the problem of leak memory with Grade class. We have to sold that with WeakKeyDictionary.

In [4]:
class Grade:
    def __init__(self):
        self._value = 0

    def __get__(self, instance, instance_type):
        return self._value

    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError(
                'Grade must be between 0 and 100')
        self._value = value

class Exam:
    # Class attributes
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

#compare the first_exam and the second_exam to see that the first and the second writing_grade reference to the same object
first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print('Writing', first_exam.writing_grade)
print('Science', first_exam.science_grade)

second_exam = Exam()
second_exam.writing_grade = 75
print(f'Second {second_exam.writing_grade} is right')
print(f'First  {first_exam.writing_grade} is wrong; '
      f'should be 82')

Writing 82
Science 99
Second 75 is right
First  75 is wrong; should be 82


In [ ]:
#example of sequence call when call instance set and get attributes
exam = Exam()
exam.writing_grade = 40
Exam.__dict__['writing_grade'].__set__(exam, 40) # __set__(instance, value)
exam.writing_grade
Exam.__dict__['writing_grade'].__get__(exam, Exam) #__get__(instance, ClassName)

In [ ]:
#the second definition of Grade
#how this second Grade definition solve the problem of reference to the same object
#because we use self.values with data structure which set and get correct instance
from weakref import WeakKeyDictionary

class Grade:
    def __init__(self):
        #WeakkyDictionary allow instance to be remove when it is not in use
        self._values = WeakKeyDictionary() #we want to have data structure which hold instance as keys

    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return self._values.get(instance, 0) #we want to  to return value of correct instance

    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError(
                'Grade must be between 0 and 100')
        self._values[instance] = value #we want to set value for correct instance
#we want to reuse __get__ and __set__ methods of Grade multiple time 
class Exam:
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

first_exam = Exam()
first_exam.writing_grade = 82
second_exam = Exam()
second_exam.writing_grade = 75
print(f'First  {first_exam.writing_grade} is right')
print(f'Second {second_exam.writing_grade} is right')

In [ ]:
#this definition will cause cause memory leak
#what is memory leak, which mean dictionary will hold all instance of do not allow garbage collection
class Grade:
    def __init__(self):
        self._values = {}

    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return self._values.get(instance, 0)

    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError(
                'Grade must be between 0 and 100')
        self._values[instance] = value
